# **a.T CUDA**

Intro to CUDA programming wiht a.T


In [1]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/'Colab Notebooks'
# this is saved in /content/drive/MyDrive/Colab Notebooks
# need to move it to Colab-Notebooks which is git checked in and make sure you
# are editing this copy

/content/drive/MyDrive/Colab Notebooks



# **Update cuda drivers**
Match nvcc to nvidia-smi

In [ ]:
%cd /content

/content


In [ ]:
!nvidia-smi

Wed Dec 31 05:22:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:

import os
print("colab_dc333.update_12_4 downgrades colab from cuda12.5 to cuda12.4 to make nvcc and nvidia-smi match")
print("this takes 5minutes, please wait ...")
os.environ['DEBIAN_FRONTEND'] = "noninteractive"
#os.system('apt-get update && apt-get install -y git')
#os.system('apt-get install -y emacs')
#os.system('apt-get install net-tools')
#os.system('apt-get install -y mlocate')
#os.system('apt-get install -y keyboard-configuration')
os.system('wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-keyring_1.1-1_all.deb')
os.system('dpkg -i cuda-keyring_1.1-1_all.deb')
os.system('apt-get update')
os.system('apt-get -y install cuda-toolkit-12-4')
os.system('rm /etc/alternatives/cuda')
os.system('ln -s  /usr/local/cuda-12.4 /etc/alternatives/cuda')
os.system('nvcc --version')


colab_dc333.update_12_4 downgrades colab from cuda12.5 to cuda12.4 to make nvcc and nvidia-smi match
this takes 5minutes, please wait ...


0

In [ ]:
# install nsys

#os.system('apt update')
os.system('apt install -y --no-install-recommends gnupg')
os.system('echo "deb http://developer.download.nvidia.com/devtools/repos/ubuntu$(source /etc/lsb-release; echo "$DISTRIB_RELEASE" | tr -d .)/$(dpkg --print-architecture) /" | tee /etc/apt/sources.list.d/nvidia-devtools.list')
os.system('apt-key adv --fetch-keys http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub')
os.system('apt update')
os.system('apt install nsight-systems-cli')
os.system('nsys status -e')

0

In [ ]:
# install ncu
!apt-get update
!apt-get install -y nvidia-nsight-compute
!ncu --version

Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 https://cli.github.com/packages stable InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:1 https://developer.download.nvidia.com/devtools/repos/ubuntu/amd64  InRelease
Err:10 https://developer.download.nvidia.com/devtools/repos/ubuntu/amd64  Release
  404  Not Found [IP: 23.0.162.178 443]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Do

Slow path transpose_gemm.cu vs fast path where transpose is implemented in stride and index changes

nvcc -O3 -lineinfo transpose_gemm.cu -o transpose_gemm -lcublas

In [ ]:
// transpose_gemm.cu
#include <cstdio>
#include <cublas_v2.h>
#include <cuda_runtime.h>

#define CHECK_CUDA(call) \
  do { \
    cudaError_t err = (call); \
    if (err != cudaSuccess) { \
      printf("CUDA error %s:%d: %s\n", __FILE__, __LINE__, cudaGetErrorString(err)); \
      exit(1); \
    } \
  } while (0)

#define CHECK_CUBLAS(call) \
  do { \
    cublasStatus_t st = (call); \
    if (st != CUBLAS_STATUS_SUCCESS) { \
      printf("cuBLAS error %s:%d: %d\n", __FILE__, __LINE__, st); \
      exit(1); \
    } \
  } while (0)

__global__ void transpose_kernel(const float* __restrict__ A,
                                 float* __restrict__ AT,
                                 int M, int N) {
  // A: (M, N) row-major
  // AT: (N, M) row-major
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;
  if (row < M && col < N) {
    AT[col * M + row] = A[row * N + col];
  }
}

int main() {
  int M = 1024;
  int K = 1024;
  int N = 1024;

  size_t bytesA = M * K * sizeof(float);
  size_t bytesB = K * N * sizeof(float);
  size_t bytesBT = N * K * sizeof(float);
  size_t bytesC = M * N * sizeof(float);

  float *hA = (float*)malloc(bytesA);
  float *hB = (float*)malloc(bytesB);

  for (int i = 0; i < M*K; ++i) hA[i] = 0.001f * i;
  for (int i = 0; i < K*N; ++i) hB[i] = 0.002f * i;

  float *dA, *dB, *dBT, *dC;
  CHECK_CUDA(cudaMalloc(&dA, bytesA));
  CHECK_CUDA(cudaMalloc(&dB, bytesB));
  CHECK_CUDA(cudaMalloc(&dBT, bytesBT));
  CHECK_CUDA(cudaMalloc(&dC, bytesC));

  CHECK_CUDA(cudaMemcpy(dA, hA, bytesA, cudaMemcpyHostToDevice));
  CHECK_CUDA(cudaMemcpy(dB, hB, bytesB, cudaMemcpyHostToDevice));

  cublasHandle_t handle;
  CHECK_CUBLAS(cublasCreate(&handle));

  dim3 block(32, 32);
  dim3 grid((K + block.x - 1) / block.x,
            (N + block.y - 1) / block.y);

  // Time with CUDA events
  cudaEvent_t start, stop;
  CHECK_CUDA(cudaEventCreate(&start));
  CHECK_CUDA(cudaEventCreate(&stop));

  CHECK_CUDA(cudaEventRecord(start));

  // 1) Explicit transpose B -> BT
  transpose_kernel<<<grid, block>>>(dB, dBT, K, N);
  CHECK_CUDA(cudaGetLastError());

  // 2) GEMM: C = A * BT
  float alpha = 1.0f, beta = 0.0f;
  // Row-major trick: treat as column-major by swapping M/N and trans flags
  //   A: (M, K)
  //   BT: (N, K) – but we want it interpreted as (K, N) column-major
  CHECK_CUBLAS(cublasSgemm(
      handle,
      CUBLAS_OP_N, CUBLAS_OP_T,  // B is effectively transposed
      N, M, K,
      &alpha,
      dBT, N,     // B^T
      dA, K,      // A
      &beta,
      dC, N));    // C

  CHECK_CUDA(cudaEventRecord(stop));
  CHECK_CUDA(cudaEventSynchronize(stop));
  float ms = 0.0f;
  CHECK_CUDA(cudaEventElapsedTime(&ms, start, stop));
  printf("Naive transpose+gemm time: %.3f ms\n", ms);

  CHECK_CUBLAS(cublasDestroy(handle));
  CHECK_CUDA(cudaFree(dA));
  CHECK_CUDA(cudaFree(dB));
  CHECK_CUDA(cudaFree(dBT));
  CHECK_CUDA(cudaFree(dC));
  free(hA);
  free(hB);
  return 0;
}

In [ ]:
// strided_gemm.cu
#include <cstdio>
#include <cublas_v2.h>
#include <cuda_runtime.h>

#define CHECK_CUDA(call)  /* same as above */
#define CHECK_CUBLAS(call) /* same as above */

int main() {
  int M = 1024;
  int K = 1024;
  int N = 1024;

  size_t bytesA = M * K * sizeof(float);
  size_t bytesB = K * N * sizeof(float);
  size_t bytesC = M * N * sizeof(float);

  float *hA = (float*)malloc(bytesA);
  float *hB = (float*)malloc(bytesB);

  for (int i = 0; i < M*K; ++i) hA[i] = 0.001f * i;
  for (int i = 0; i < K*N; ++i) hB[i] = 0.002f * i;

  float *dA, *dB, *dC;
  CHECK_CUDA(cudaMalloc(&dA, bytesA));
  CHECK_CUDA(cudaMalloc(&dB, bytesB));
  CHECK_CUDA(cudaMalloc(&dC, bytesC));

  CHECK_CUDA(cudaMemcpy(dA, hA, bytesA, cudaMemcpyHostToDevice));
  CHECK_CUDA(cudaMemcpy(dB, hB, bytesB, cudaMemcpyHostToDevice));

  cublasHandle_t handle;
  CHECK_CUBLAS(cublasCreate(&handle));

  cudaEvent_t start, stop;
  CHECK_CUDA(cudaEventCreate(&start));
  CHECK_CUDA(cudaEventCreate(&stop));

  CHECK_CUDA(cudaEventRecord(start));

  float alpha = 1.0f, beta = 0.0f;

  // C = A * B^T, but we NEVER transpose B in memory.
  // We just tell cuBLAS: opB = CUBLAS_OP_T
  CHECK_CUBLAS(cublasSgemm(
      handle,
      CUBLAS_OP_N, CUBLAS_OP_T,  // A normal, B interpreted transposed
      N, M, K,
      &alpha,
      dB, N,     // B, but read as transposed
      dA, K,     // A
      &beta,
      dC, N));   // C

  CHECK_CUDA(cudaEventRecord(stop));
  CHECK_CUDA(cudaEventSynchronize(stop));
  float ms = 0.0f;
  CHECK_CUDA(cudaEventElapsedTime(&ms, start, stop));
  printf("Direct cuBLAS (opB=T) time: %.3f ms\n", ms);

  CHECK_CUBLAS(cublasDestroy(handle));
  CHECK_CUDA(cudaFree(dA));
  CHECK_CUDA(cudaFree(dB));
  CHECK_CUDA(cudaFree(dC));
  free(hA);
  free(hB);
  return 0;
}

/content# ./strided_gemm
Direct cuBLAS (opB=T) time: 0.000 ms
/content# ./transpose_gemm
Naive transpose+gemm time: 6.761 ms


In [ ]:
/#include <cstdio>
#include <cuda_runtime.h>

#define TILE_DIM 32
#define BLOCK_ROWS 8

// ====================== Naive kernel ======================

__global__ void transpose_naive(const float* __restrict__ in,
                                float* __restrict__ out,
                                int H, int W) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < H && col < W) {
        out[col * H + row] = in[row * W + col];
    }
}

// ====================== Tiled kernel ======================

__global__ void transpose_tiled(const float* __restrict__ in,
                                float* __restrict__ out,
                                int H, int W) {
    __shared__ float tile[TILE_DIM][TILE_DIM + 1]; // +1 to avoid bank conflicts

    int x = blockIdx.x * TILE_DIM + threadIdx.x;  // column index in input
    int y = blockIdx.y * TILE_DIM + threadIdx.y;  // row index in input

    // Load tile from global memory into shared memory
    for (int i = 0; i < TILE_DIM; i += BLOCK_ROWS) {
        int yy = y + i;
        if (x < W && yy < H) {
            tile[threadIdx.y + i][threadIdx.x] = in[yy * W + x];
        }
    }

    __syncthreads();

    // Transpose tile and write back to global memory
    x = blockIdx.y * TILE_DIM + threadIdx.x;  // transposed col index
    y = blockIdx.x * TILE_DIM + threadIdx.y;  // transposed row index

    for (int i = 0; i < TILE_DIM; i += BLOCK_ROWS) {
        int yy = y + i;
        if (x < H && yy < W) {
            out[yy * H + x] = tile[threadIdx.x][threadIdx.y + i];
        }
    }
}

// ====================== Benchmark main ======================

int main() {
    const int H = 4096;
    const int W = 4096;
    size_t bytes = H * W * sizeof(float);

    float *d_in, *d_out;
    cudaMalloc(&d_in, bytes);
    cudaMalloc(&d_out, bytes);

    cudaMemset(d_in, 1, bytes);
    cudaMemset(d_out, 0, bytes);

    dim3 block1(16, 16);
    dim3 grid1((W + block1.x - 1) / block1.x,
               (H + block1.y - 1) / block1.y);

    dim3 block2(TILE_DIM, BLOCK_ROWS);
    dim3 grid2((W + TILE_DIM - 1) / TILE_DIM,
               (H + TILE_DIM - 1) / TILE_DIM);

    // Warmup both kernels
    for (int i = 0; i < 5; ++i) {
        transpose_naive<<<grid1, block1>>>(d_in, d_out, H, W);
        transpose_tiled<<<grid2, block2>>>(d_in, d_out, H, W);
    }
    cudaDeviceSynchronize();

    // Time naive
    float ms_naive;
    {
        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);

        cudaEventRecord(start);
        for (int i = 0; i < 50; ++i) {
            transpose_naive<<<grid1, block1>>>(d_in, d_out, H, W);
        }
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&ms_naive, start, stop);

        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    // Time tiled
    float ms_tiled;
    {
        cudaEvent_t start, stop;
        cudaEventCreate(&start);
        cudaEventCreate(&stop);

        cudaEventRecord(start);
        for (int i = 0; i < 50; ++i) {
            transpose_tiled<<<grid2, block2>>>(d_in, d_out, H, W);
        }
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&ms_tiled, start, stop);

        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    printf("Naive transpose:  %f ms (avg over 50)\n", ms_naive / 50.0f);
    printf("Tiled transpose:  %f ms (avg over 50)\n", ms_tiled / 50.0f);

    cudaFree(d_in);
    cudaFree(d_out);
    return 0;
}

/content# ./transpose_bench
Naive transpose:  1.234682 ms (avg over 50)
Tiled transpose:  0.669043 ms (avg over 50)

In [ ]:
nsys profile \
  -t cuda,osrt \
  -o transpose_report \
  ./transpose_bench

In [ ]:
ncu --set full --page raw --csv ./transpose_bench > transpose_stall.csv

In [ ]:
#!/usr/bin/env python3
import csv
import json
import sys

if len(sys.argv) != 3:
    print("usage: python csv_to_json.py input.csv output.json")
    sys.exit(1)

inp, out = sys.argv[1], sys.argv[2]

metrics = {}

with open(inp, newline="") as f:
    reader = csv.reader(f)

    name_idx = None
    value_idx = None

    for row in reader:
        if not row:
            continue

        # Detect header rows for metric tables
        # Typical Nsight Compute CSV metric header looks like:
        # ID, "Metric Name", "Metric Description", "Metric Unit", "Metric Value"
        if "Metric Name" in row and "Metric Value" in row:
            name_idx = row.index("Metric Name")
            value_idx = row.index("Metric Value")
            continue

        # If we don't have indices yet, skip
        if name_idx is None or value_idx is None:
            continue

        if len(row) <= max(name_idx, value_idx):
            continue

        metric_name = row[name_idx].strip()
        metric_val_str = row[value_idx].strip()

        try:
            val = float(metric_val_str)
        except ValueError:
            continue

        metrics[metric_name] = val


def get(name: str, default: float = 0.0) -> float:
    return float(metrics.get(name, default))


# Build the JSON structure used by the React NsightProfileCharts component
data = {
    "kernelName": "transpose_stall",  # you can override this later if you like
    "occupancy": {
        "theoretical": get("sm__maximum_warps_active.avg.pct_of_peak_sustained_active"),
        "achieved":    get("sm__warps_active.avg.pct_of_peak_sustained_active"),
    },
    "throughput": {
        "compute": get("sm__throughput.avg.pct_of_peak_sustained_elapsed"),
        "memory":  get("lts__throughput.avg.pct_of_peak_sustained_elapsed"),
        "dram":    get("dram__throughput.avg.pct_of_peak_sustained_elapsed"),
    },
    # Still leaving stall diagnostics as 0.0 for now – those live in the
    # "Diagnostics" / "SpeedOfLight" text sections, not simple numeric metrics.
    "stalls": {
        "uncoalescedGlobalAccesses": 0.0,
        "l1StorePattern": 0.0,
        "longScoreboard": 0.0,
    },
}

with open(out, "w") as f:
    json.dump(data, f, indent=2)

print("Wrote", out)

In [ ]:
mean_c = segment_mean(vals_flat, seg_ids, K=C)[c]

In [ ]:
# segment mean using scater
scatter_mean_1d(vals, seg_ids, K)  # using index_add_ (scatter)

In [ ]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F


# ---------- Scatter / segment-mean primitives ----------

def scatter_mean_1d(vals: torch.Tensor, seg_ids: torch.Tensor, K: int) -> torch.Tensor:
    """
    vals:    (N,)  1D tensor of values
    seg_ids: (N,)  int64/int32 segment IDs in [0, K-1]
    K:       number of segments

    Returns: (K,) mean per segment.
    """
    sums = vals.new_zeros(K)
    counts = vals.new_zeros(K, dtype=torch.long)

    # sums[k]   += vals[i] for all i where seg_ids[i] == k
    # counts[k] += 1       for all i where seg_ids[i] == k
    sums.index_add_(0, seg_ids, vals)
    counts.index_add_(0, seg_ids, torch.ones_like(seg_ids, dtype=torch.long))

    return sums / counts.clamp_min(1).to(sums.dtype)


class ScatterBatchNorm1d(nn.Module):
    """
    A BatchNorm1d analog that computes per-channel mean/var
    using generic scatter/segment-mean primitives.

    Input: (N, C)  (we'll flatten (B, S, D) -> (B*S, D) before calling).
    """

    def __init__(self, num_features: int, eps: float = 1e-5, affine: bool = True):
        super().__init__()
        self.num_features = num_features
        self.eps = eps
        self.affine = affine

        if affine:
            self.weight = nn.Parameter(torch.ones(num_features))
            self.bias = nn.Parameter(torch.zeros(num_features))
        else:
            self.register_parameter("weight", None)
            self.register_parameter("bias", None)

        # For benchmarking / training only, we skip running_mean/var
        # and track_running_stats logic to keep the code focused on the
        # scatter-based reduction.

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: (N, C)
        """
        N, C = x.shape
        assert C == self.num_features

        # Flatten to scalars
        vals = x.reshape(-1)               # (N*C,)
        # Segment IDs: for each row n, features 0..C-1
        # seg_ids[k] = feature index for that scalar
        seg_ids = torch.arange(C, device=x.device).repeat(N)  # (N*C,)
        K = C

        # Compute per-channel mean and second moment via scatter mean
        mean = scatter_mean_1d(vals, seg_ids, K)        # (C,)
        mean2 = scatter_mean_1d(vals * vals, seg_ids, K)  # (C,)

        var = mean2 - mean * mean  # (C,)

        mean = mean.view(1, C)
        var = var.view(1, C)

        y = (x - mean) / torch.sqrt(var + self.eps)

        if self.affine:
            y = y * self.weight.view(1, C) + self.bias.view(1, C)

        return y


# ---------- Mini Transformer blocks ----------

class MiniTransformerBlockBN(nn.Module):
    """
    Mini Transformer block using standard nn.BatchNorm1d
    on the (B*S, D) flattened representation.
    """

    def __init__(self, d_model=64, nhead=4, dim_ff=128, dropout=0.1):
        super().__init__()
        self.attn = nn.MultiheadAttention(d_model, nhead, batch_first=True)
        self.bn1 = nn.BatchNorm1d(d_model, affine=True, track_running_stats=False)
        self.bn2 = nn.BatchNorm1d(d_model, affine=True, track_running_stats=False)

        self.linear1 = nn.Linear(d_model, dim_ff)
        self.linear2 = nn.Linear(dim_ff, d_model)

        self.dropout = nn.Dropout(dropout)
        self.activation = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: (B, S, D)
        """
        B, S, D = x.shape

        # Self-attention
        attn_out, _ = self.attn(x, x, x, need_weights=False)
        x = x + attn_out

        # BN over (B*S, D)
        x_flat = x.view(B * S, D)
        x_flat = self.bn1(x_flat)
        x = x_flat.view(B, S, D)

        # FFN
        y = self.linear2(self.dropout(self.activation(self.linear1(x))))
        x = x + y

        # BN again
        x_flat = x.view(B * S, D)
        x_flat = self.bn2(x_flat)
        x = x_flat.view(B, S, D)

        return x


class MiniTransformerBlockScatterBN(nn.Module):
    """
    Mini Transformer block using ScatterBatchNorm1d
    instead of nn.BatchNorm1d.
    """

    def __init__(self, d_model=64, nhead=4, dim_ff=128, dropout=0.1):
        super().__init__()
        self.attn = nn.MultiheadAttention(d_model, nhead, batch_first=True)
        self.sbn1 = ScatterBatchNorm1d(d_model, affine=True)
        self.sbn2 = ScatterBatchNorm1d(d_model, affine=True)

        self.linear1 = nn.Linear(d_model, dim_ff)
        self.linear2 = nn.Linear(dim_ff, d_model)

        self.dropout = nn.Dropout(dropout)
        self.activation = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: (B, S, D)
        """
        B, S, D = x.shape

        # Self-attention
        attn_out, _ = self.attn(x, x, x, need_weights=False)
        x = x + attn_out

        # Scatter-based BN over (B*S, D)
        x_flat = x.view(B * S, D)
        x_flat = self.sbn1(x_flat)
        x = x_flat.view(B, S, D)

        # FFN
        y = self.linear2(self.dropout(self.activation(self.linear1(x))))
        x = x + y

        # Scatter-based BN again
        x_flat = x.view(B * S, D)
        x_flat = self.sbn2(x_flat)
        x = x_flat.view(B, S, D)

        return x


# ---------- Simple benchmark ----------

def benchmark_block(
    block: nn.Module,
    name: str,
    device: str = "cuda",
    B: int = 32,
    S: int = 128,
    D: int = 64,
    iters: int = 50,
    warmup: int = 10,
):
    block = block.to(device)
    block.train()
    x = torch.randn(B, S, D, device=device)
    opt = torch.optim.Adam(block.parameters(), lr=1e-3)

    # Warmup
    for _ in range(warmup):
        opt.zero_grad(set_to_none=True)
        y = block(x)
        loss = y.pow(2).mean()
        loss.backward()
        opt.step()

    # Timing (forward + backward)
    if device.startswith("cuda"):
        torch.cuda.synchronize()
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)

        start.record()
        for _ in range(iters):
            opt.zero_grad(set_to_none=True)
            y = block(x)
            loss = y.pow(2).mean()
            loss.backward()
            opt.step()
        end.record()
        torch.cuda.synchronize()
        elapsed_ms = start.elapsed_time(end) / iters
    else:
        t0 = time.time()
        for _ in range(iters):
            opt.zero_grad(set_to_none=True)
            y = block(x)
            loss = y.pow(2).mean()
            loss.backward()
            opt.step()
        t1 = time.time()
        elapsed_ms = (t1 - t0) * 1000 / iters

    print(f"{name}: {elapsed_ms:.3f} ms/iter "
          f"(B={B}, S={S}, D={D}, device={device})")


if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using device:", device)

    # Quick sanity check: ScatterBatchNorm1d ~= BatchNorm1d (train-mode)
    N, C = 8, 16
    x_test = torch.randn(N, C, device=device)
    bn = nn.BatchNorm1d(C, affine=True, track_running_stats=False).to(device)
    sbn = ScatterBatchNorm1d(C, affine=True).to(device)
    with torch.no_grad():
        sbn.weight.copy_(bn.weight)
        sbn.bias.copy_(bn.bias)
    y1 = bn(x_test)
    y2 = sbn(x_test)
    print("Max diff BN vs ScatterBN (sanity check):",
          (y1 - y2).abs().max().item())

    # Benchmark the two transformer blocks
    benchmark_block(MiniTransformerBlockBN(), "MiniTransformer + BatchNorm", device=device)
    benchmark_block(MiniTransformerBlockScatterBN(), "MiniTransformer + ScatterMeanBN", device=device)

Using device: cuda
Max diff BN vs ScatterBN (sanity check): 3.5762786865234375e-07
MiniTransformer + BatchNorm: 2.922 ms/iter (B=32, S=128, D=64, device=cuda)
MiniTransformer + ScatterMeanBN: 4.801 ms/iter (B=32, S=128, D=64, device=cuda)


In [ ]:
# karpathy replicating medians in (B,T,C) format with triangular matrix multiply
import torch
torch.manual_seed(42)
a = torch.ones(3,3)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('----')
print('b=')
print(b)
print('----')
print('c=')
print(c)
print('---end multiply w identiy matrix---')

# can see matrix multiply with Identiy matrix and data produces sums in the columns. Column sums 2+6+6=16, 7+4+5=16
# the matrix multiply is a sum when we take the dot product. First row [1,1,1] * first col [2,6,6] gives sum 2+6+6=14,

# second step take the lower triangular matrix instead if Identity matrix. This adds 0s
print('---replace with lower triangular and make rows sum to 1---')
print('  ')
a = torch.tril(torch.ones(3,3))/torch.sum(a,1,keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('----')
print('b=')
print(b)
print('----')
print('c=')
print(c)
print('---end multiply w lower triagular rows normalzied to sum 1---')


a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
----
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
----
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])
---end multiply w identiy matrix---
---replace with lower triangular and make rows sum to 1---
  
a=
tensor([[0.3333, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.0000],
        [0.3333, 0.3333, 0.3333]])
----
b=
tensor([[0., 4.],
        [0., 3.],
        [8., 4.]])
----
c=
tensor([[0.0000, 1.3333],
        [0.0000, 2.3333],
        [2.6667, 3.6667]])
---end multiply w lower triagular rows normalzied to sum 1---


In [ ]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

try:
    import triton
    import triton.language as tl
    HAS_TRITON = True
except ImportError:
    HAS_TRITON = False
    print("Triton not found; Triton-based ScatterBN will be disabled.")


# ============================================================
#  Scatter / segment-mean primitives (PyTorch + Triton)
# ============================================================

def scatter_mean_1d(vals: torch.Tensor, seg_ids: torch.Tensor, K: int) -> torch.Tensor:
    """
    Generic segment-mean using PyTorch's index_add_.

    vals:    (N,)
    seg_ids: (N,) int64/int32
    K:       number of segments

    Returns: (K,) segment means.
    """
    sums = vals.new_zeros(K)
    counts = vals.new_zeros(K, dtype=torch.long)

    sums.index_add_(0, seg_ids, vals)
    counts.index_add_(0, seg_ids, torch.ones_like(seg_ids, dtype=torch.long))

    return sums / counts.clamp_min(1).to(sums.dtype)


# ---------------- Triton kernel for per-channel stats ----------------

if HAS_TRITON:
    @triton.jit
    def bn_stats_triton_kernel(
        x_ptr,        # *f32, shape (N, C), row-major
        mean_ptr,     # *f32, shape (C,)
        var_ptr,      # *f32, shape (C,)
        N, C,
        BLOCK_N: tl.constexpr,
    ):
        """
        Each program handles one channel (feature dimension).
        We compute mean and variance across the N dimension for that channel.

        x is assumed contiguous of shape (N, C).
        """

        c = tl.program_id(0)  # channel index
        if c >= C:
            return

        # Accumulators in fp32
        sum_x = tl.zeros((), dtype=tl.float32)
        sum_x2 = tl.zeros((), dtype=tl.float32)

        # Loop over rows in tiles of BLOCK_N
        # (We manually unroll over N dimension)
        n = 0
        while n < N:
            offs_n = n + tl.arange(0, BLOCK_N)
            mask = offs_n < N

            # x index = n * C + c (row-major)
            idx = offs_n * C + c
            x_vals = tl.load(x_ptr + idx, mask=mask, other=0.0)
            x_vals = x_vals.to(tl.float32)

            sum_x += tl.sum(x_vals, axis=0)
            sum_x2 += tl.sum(x_vals * x_vals, axis=0)

            n += BLOCK_N

        n_f = tl.full((), N, dtype=tl.float32)
        mean = sum_x / n_f
        ex2 = sum_x2 / n_f
        var = ex2 - mean * mean

        tl.store(mean_ptr + c, mean)
        tl.store(var_ptr + c, var)


    def bn_stats_triton(x: torch.Tensor):
        """
        x: (N, C) float32, contiguous

        Returns:
            mean: (C,)
            var:  (C,)
        """
        assert x.ndim == 2
        assert x.is_cuda
        assert x.dtype == torch.float32
        x = x.contiguous()
        N, C = x.shape

        mean = torch.empty(C, device=x.device, dtype=torch.float32)
        var = torch.empty(C, device=x.device, dtype=torch.float32)

        BLOCK_N = 256
        grid = (C,)

        bn_stats_triton_kernel[grid](
            x, mean, var,
            N, C,
            BLOCK_N=BLOCK_N,
        )

        return mean, var


# ============================================================
#  Normalization modules
# ============================================================

class ScatterBatchNorm1d(nn.Module):
    """
    BatchNorm1d implemented via generic scatter/segment-mean using PyTorch ops.

    Input: (N, C)
    """

    def __init__(self, num_features: int, eps: float = 1e-5, affine: bool = True):
        super().__init__()
        self.num_features = num_features
        self.eps = eps
        self.affine = affine

        if affine:
            self.weight = nn.Parameter(torch.ones(num_features))
            self.bias = nn.Parameter(torch.zeros(num_features))
        else:
            self.register_parameter("weight", None)
            self.register_parameter("bias", None)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: (N, C)
        """
        N, C = x.shape
        assert C == self.num_features

        vals = x.reshape(-1)  # (N*C,)

        # segment ids: feature index for each scalar
        seg_ids = torch.arange(C, device=x.device).repeat(N)  # (N*C,)
        K = C

        mean = scatter_mean_1d(vals, seg_ids, K)                # (C,)
        mean2 = scatter_mean_1d(vals * vals, seg_ids, K)        # (C,)
        var = mean2 - mean * mean                               # (C,)

        mean = mean.view(1, C)
        var = var.view(1, C)

        y = (x - mean) / torch.sqrt(var + self.eps)

        if self.affine:
            y = y * self.weight.view(1, C) + self.bias.view(1, C)

        return y


class ScatterBatchNorm1dTriton(nn.Module):
    """
    BatchNorm1d-like layer whose per-channel stats are computed via a Triton kernel.

    Input: (N, C)
    """

    def __init__(self, num_features: int, eps: float = 1e-5, affine: bool = True):
        super().__init__()
        if not HAS_TRITON:
            raise RuntimeError("Triton is not available but ScatterBatchNorm1dTriton was instantiated.")

        self.num_features = num_features
        self.eps = eps
        self.affine = affine

        if affine:
            self.weight = nn.Parameter(torch.ones(num_features))
            self.bias = nn.Parameter(torch.zeros(num_features))
        else:
            self.register_parameter("weight", None)
            self.register_parameter("bias", None)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: (N, C), float32, CUDA
        """
        assert x.is_cuda
        assert x.dtype == torch.float32

        N, C = x.shape
        assert C == self.num_features

        mean, var = bn_stats_triton(x)  # (C,), (C,)

        mean = mean.view(1, C)
        var = var.view(1, C)

        y = (x - mean) / torch.sqrt(var + self.eps)

        if self.affine:
            y = y * self.weight.view(1, C) + self.bias.view(1, C)

        return y


# ============================================================
#  Mini Transformer blocks
# ============================================================

class MiniTransformerBlockBN(nn.Module):
    """
    Mini Transformer block with standard nn.BatchNorm1d.
    """

    def __init__(self, d_model=64, nhead=4, dim_ff=128, dropout=0.1):
        super().__init__()
        self.attn = nn.MultiheadAttention(d_model, nhead, batch_first=True)
        self.bn1 = nn.BatchNorm1d(d_model, affine=True, track_running_stats=False)
        self.bn2 = nn.BatchNorm1d(d_model, affine=True, track_running_stats=False)

        self.linear1 = nn.Linear(d_model, dim_ff)
        self.linear2 = nn.Linear(dim_ff, d_model)

        self.dropout = nn.Dropout(dropout)
        self.activation = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: (B, S, D)
        """
        B, S, D = x.shape

        attn_out, _ = self.attn(x, x, x, need_weights=False)
        x = x + attn_out

        x_flat = x.view(B * S, D)
        x_flat = self.bn1(x_flat)
        x = x_flat.view(B, S, D)

        y = self.linear2(self.dropout(self.activation(self.linear1(x))))
        x = x + y

        x_flat = x.view(B * S, D)
        x_flat = self.bn2(x_flat)
        x = x_flat.view(B, S, D)

        return x


class MiniTransformerBlockScatterBN(nn.Module):
    """
    Mini Transformer block using ScatterBatchNorm1d (PyTorch scatter).
    """

    def __init__(self, d_model=64, nhead=4, dim_ff=128, dropout=0.1):
        super().__init__()
        self.attn = nn.MultiheadAttention(d_model, nhead, batch_first=True)
        self.sbn1 = ScatterBatchNorm1d(d_model, affine=True)
        self.sbn2 = ScatterBatchNorm1d(d_model, affine=True)

        self.linear1 = nn.Linear(d_model, dim_ff)
        self.linear2 = nn.Linear(dim_ff, d_model)

        self.dropout = nn.Dropout(dropout)
        self.activation = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, S, D = x.shape

        attn_out, _ = self.attn(x, x, x, need_weights=False)
        x = x + attn_out

        x_flat = x.view(B * S, D)
        x_flat = self.sbn1(x_flat)
        x = x_flat.view(B, S, D)

        y = self.linear2(self.dropout(self.activation(self.linear1(x))))
        x = x + y

        x_flat = x.view(B * S, D)
        x_flat = self.sbn2(x_flat)
        x = x_flat.view(B, S, D)

        return x


class MiniTransformerBlockScatterBNTriton(nn.Module):
    """
    Mini Transformer block using ScatterBatchNorm1dTriton.
    """

    def __init__(self, d_model=64, nhead=4, dim_ff=128, dropout=0.1):
        super().__init__()
        if not HAS_TRITON:
            raise RuntimeError("Triton not available; cannot create Triton-based block.")

        self.attn = nn.MultiheadAttention(d_model, nhead, batch_first=True)
        self.sbn1 = ScatterBatchNorm1dTriton(d_model, affine=True)
        self.sbn2 = ScatterBatchNorm1dTriton(d_model, affine=True)

        self.linear1 = nn.Linear(d_model, dim_ff)
        self.linear2 = nn.Linear(dim_ff, d_model)

        self.dropout = nn.Dropout(dropout)
        self.activation = nn.ReLU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, S, D = x.shape

        attn_out, _ = self.attn(x, x, x, need_weights=False)
        x = x + attn_out

        x_flat = x.view(B * S, D)
        x_flat = self.sbn1(x_flat)
        x = x_flat.view(B, S, D)

        y = self.linear2(self.dropout(self.activation(self.linear1(x))))
        x = x + y

        x_flat = x.view(B * S, D)
        x_flat = self.sbn2(x_flat)
        x = x_flat.view(B, S, D)

        return x


# ============================================================
#  Benchmarking utilities
# ============================================================

def benchmark_block(
    block: nn.Module,
    name: str,
    device: str = "cuda",
    B: int = 32,
    S: int = 128,
    D: int = 64,
    iters: int = 50,
    warmup: int = 10,
):
    block = block.to(device)
    block.train()
    x = torch.randn(B, S, D, device=device, dtype=torch.float32)
    opt = torch.optim.Adam(block.parameters(), lr=1e-3)

    # Warmup
    for _ in range(warmup):
        opt.zero_grad(set_to_none=True)
        y = block(x)
        loss = y.pow(2).mean()
        loss.backward()
        opt.step()

    # Timing (forward + backward)
    if device.startswith("cuda"):
        torch.cuda.synchronize()
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)

        start.record()
        for _ in range(iters):
            opt.zero_grad(set_to_none=True)
            y = block(x)
            loss = y.pow(2).mean()
            loss.backward()
            opt.step()
        end.record()
        torch.cuda.synchronize()
        elapsed_ms = start.elapsed_time(end) / iters
    else:
        t0 = time.time()
        for _ in range(iters):
            opt.zero_grad(set_to_none=True)
            y = block(x)
            loss = y.pow(2).mean()
            loss.backward()
            opt.step()
        t1 = time.time()
        elapsed_ms = (t1 - t0) * 1000 / iters

    print(f"{name}: {elapsed_ms:.3f} ms/iter "
          f"(B={B}, S={S}, D={D}, device={device})")


if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using device:", device)

    # Sanity check for PyTorch ScatterBN vs BatchNorm1d
    N, C = 8, 16
    x_test = torch.randn(N, C, device=device)
    bn = nn.BatchNorm1d(C, affine=True, track_running_stats=False).to(device)
    sbn = ScatterBatchNorm1d(C, affine=True).to(device)
    with torch.no_grad():
        sbn.weight.copy_(bn.weight)
        sbn.bias.copy_(bn.bias)
    y1 = bn(x_test)
    y2 = sbn(x_test)
    print("Max diff BN vs ScatterBN(PyTorch):",
          (y1 - y2).abs().max().item())

    # If Triton is available, sanity check it too
    if HAS_TRITON and device == "cuda":
        sbn_t = ScatterBatchNorm1dTriton(C, affine=True).to(device)
        with torch.no_grad():
            sbn_t.weight.copy_(bn.weight)
            sbn_t.bias.copy_(bn.bias)
        y3 = sbn_t(x_test)
        print("Max diff BN vs ScatterBN(Triton):",
              (y1 - y3).abs().max().item())

    print("\n=== Benchmarking mini Transformer blocks ===")
    benchmark_block(MiniTransformerBlockBN(), "MiniTransformer + BatchNorm", device=device)
    benchmark_block(MiniTransformerBlockScatterBN(), "MiniTransformer + ScatterBN(PyTorch)", device=device)

    if HAS_TRITON and device == "cuda":
        benchmark_block(MiniTransformerBlockScatterBNTriton(), "MiniTransformer + ScatterBN(Triton)", device=device)
    else:
        print("Skipping Triton-based block benchmark (no Triton or no CUDA).")

Using device: cuda
Max diff BN vs ScatterBN(PyTorch): 4.76837158203125e-07
Max diff BN vs ScatterBN(Triton): 2.384185791015625e-07

=== Benchmarking mini Transformer blocks ===
MiniTransformer + BatchNorm: 4.074 ms/iter (B=32, S=128, D=64, device=cuda)
MiniTransformer + ScatterBN(PyTorch): 4.648 ms/iter (B=32, S=128, D=64, device=cuda)
MiniTransformer + ScatterBN(Triton): 3.073 ms/iter (B=32, S=128, D=64, device=cuda)


In [ ]:
x=torch.tensor([[1,2,3],[4,5,6]])
print(x, x.shape)
#y = x/torch.sum(x,dim=1)
print(torch.sum(x, dim=0, keepdim=True))
print(torch.sum(x, dim=0, keepdim=False))
print(torch.sum(x, dim=1, keepdim=True))
print(torch.sum(x, dim=1, keepdim=False))
#print(s)
print(torch.sum(x))

tensor([[1, 2, 3],
        [4, 5, 6]]) torch.Size([2, 3])
tensor([[5, 7, 9]])
tensor([5, 7, 9])
tensor([[ 6],
        [15]])
tensor([ 6, 15])
tensor([[3],
        [7]])
tensor(21)


# **Matrix and Vector multiply for averages.**

There are different versions of averages which are used for normalization. Karpathy develops the t-1 or the autoregresseive average where the average of the sequence of a,b,c,d has 4 averages; at t=0, avg=a, t=1 avg=(a+b)/2, t=2 avg=
(a+b+c)/3, etc...


<ul>
<li>$v = \frac{1}{N}[1,1,1,...]$ where num ones = N</li>
<li>A cumulative average is the conventional average $\frac{1}{N}\sum_0^{N-1}x_i$. It is time invariant. Shifting the sequence produces the same average. $v=[1,1,1...len(x)]$ and the data is $x$. cumulative avg = $v^T@x$</li>
<li>A column average $v^T@X$</li>
<li>How to derive row and column avg. X=(B,D). Create a 2x3 test matrix
[[1,2,3],[4,5,6]]. Make sure the TM is not symmetric to reduce confusion. We have to options an identity matrix 1x2 if I@X or 3x1 if X@I. There are 2 rows so a row sum must have 2 rows so you know 1x2 is rows and there are 3 cols and you know you need 3 columns for a column sum. so I@X is row sum and X@I is column sum. Then add 1/N to get avg. N=num elements in row or col.
</li>
<li>A row average $X@v$</li>
<li>Weighted avg for softmax. W=(T), V=(T,D). $avg=W^T@V$ Output = (D,) or (D,1) if keepdims=True. Because avg collapases and removes dimensions by default</li>
<li>How to derive Weighted Softmax Avg</li>
<li>Sequence avg: <li>
<li>How to derive sequence avg. </li>
</ul>

In [ ]:
import torch

x = torch.tensor([1,2,3,4,5,6]).float()

v = (1/6)*(torch.ones(6))
print("avg of sum of all elements v@x:",v@x)
tri = torch.tril(torch.ones(6,6))
avg = (tri@x)/torch.arange(1,7)

print("sums:",tri@x)
print("torch arange:",torch.arange(1,7))
print("rolling avg:",avg) # 1/1, (1+2)/2, (1+2+3)/3,...

avg of sum of all elements v@x: tensor(3.5000)
sums: tensor([ 1.,  3.,  6., 10., 15., 21.])
torch arange: tensor([1, 2, 3, 4, 5, 6])
avg: tensor([1.0000, 1.5000, 2.0000, 2.5000, 3.0000, 3.5000])


In [ ]:
import torch
import numpy as np
from typing import Optional, Union

# avoid division by 0
def safe_torch_mean(x: torch.Tensor) -> torch.Tensor:
    return x.float().sum() / max(x.numel(), 1)

def safe_mean(
    x: torch.Tensor,
    dim: Optional[int] = None,
    keepdim: bool = False,
    default: Union[float, int] = 0.0,
) -> torch.Tensor:
    """
    Mean that:
      - casts non-floating dtypes to float32
      - returns `default` when there are no elements along `dim`
    """
    if not x.is_floating_point():
        x = x.to(torch.float32)

    if dim is None:
        if x.numel() == 0:
            # scalar default
            return x.new_tensor(float(default))
        return x.mean()

    # Mean along a dimension
    if x.size(dim) == 0:
        # build output shape manually
        out_shape = list(x.shape)
        if keepdim:
            out_shape[dim] = 1
        else:
            del out_shape[dim]
        return x.new_full(out_shape, float(default))

    return x.mean(dim=dim, keepdim=keepdim)

x = torch.tensor([], dtype=torch.float32)
print(safe_mean(x))  # tensor(0.)

x = torch.randint(0, 10, (3,), dtype=torch.int8)
print(safe_mean(x))  # float32 mean, no error


def masked_mean(
    x: torch.Tensor,
    mask: torch.Tensor,
    dim: int,
    keepdim: bool = False,
    default: Union[float, int] = 0.0,
) -> torch.Tensor:
    """
    Mean over elements where mask == 1/True along `dim`.

    x: (..., D, ...)
    mask: same shape as x or broadcastable to x
    """
    if not x.is_floating_point():
        x = x.to(torch.float32)

    # make mask float for multiplication
    m = mask.to(x.dtype)
    # broadcast OK: this relies on PyTorch broadcasting
    masked_x = x * m

    # sum over dim
    num = masked_x.sum(dim=dim, keepdim=keepdim)
    den = m.sum(dim=dim, keepdim=keepdim)

    # safe division: where den > 0, num / den; else default
    default_tensor = num.new_full(num.shape, float(default))
    mean = torch.where(den > 0, num / torch.clamp(den, min=1e-12), default_tensor)
    return mean

x = torch.tensor([[1., 2., 3.],
                  [4., 5., 6.]])
mask = torch.tensor([[1, 0, 1],
                     [0, 0, 0]])  # second row all masked out

print(masked_mean(x, mask, dim=1))
# tensor([2., 0.])  (last row default=0)

print(masked_mean(x, mask, dim=1, default=-1.0))
# tensor([2., -1.])


def segment_mean(
    values: torch.Tensor,
    segment_ids: torch.Tensor,
    num_segments: Optional[int] = None,
    default: Union[float, int] = 0.0,
) -> torch.Tensor:
    """
    Compute mean over segments:
        segment_means[k] = mean(values[segment_ids == k])

    values: (N, D) or (N,)
    segment_ids: (N,) ints in [0, num_segments-1]
    """
    if not values.is_floating_point():
        values = values.to(torch.float32)

    if values.dim() == 1:
        values = values.unsqueeze(-1)  # make it (N, 1)

    N, D = values.shape
    segment_ids = segment_ids.to(torch.long)

    if num_segments is None:
        num_segments = int(segment_ids.max().item()) + 1 if N > 0 else 0

    device = values.device
    dtype = values.dtype

    # sums for each segment
    sums = torch.zeros(num_segments, D, device=device, dtype=dtype)
    counts = torch.zeros(num_segments, 1, device=device, dtype=dtype)

    # index_add along segment dimension
    sums.index_add_(0, segment_ids, values)
    counts.index_add_(0, segment_ids, torch.ones_like(values[:, :1]))

    default_tensor = sums.new_full(sums.shape, float(default))
    means = torch.where(
        counts > 0,
        sums / torch.clamp(counts, min=1e-12),
        default_tensor,
    )

    # squeeze if original was 1D
    if values.shape[1] == 1:
        means = means.squeeze(-1)

    return means

vals = torch.tensor([[1., 2.],
                     [3., 4.],
                     [10., 20.]], dtype=torch.float32)
seg = torch.tensor([0, 0, 2])   # segment 1 is empty

print(segment_mean(vals, seg, num_segments=3, default=0.0))
# tensor([[2., 3.],      # mean of rows 0 and 1
#         [0., 0.],      # empty segment -> default
#         [10., 20.]])   # row 2

def batch_safe_mean(
    x: torch.Tensor,
    mask: torch.Tensor,
    default: Union[float, int] = 0.0,
) -> torch.Tensor:
    """
    Per-batch masked mean over time dimension 1.

    x: (B, T, D) or (B, T)
    mask: (B, T) with 1/True = valid entries
    """
    if x.dim() == 2:
        # (B, T) -> (B, T, 1) so we reuse the same logic
        x = x.unsqueeze(-1)
        squeeze_back = True
    else:
        squeeze_back = False

    # broadcast mask to (B, T, 1)
    mask_exp = mask.unsqueeze(-1)

    means = masked_mean(
        x,
        mask_exp,
        dim=1,           # average over time
        keepdim=False,
        default=default,
    )

    if squeeze_back:
        means = means.squeeze(-1)

    return means

B, T, D = 2, 5, 3
x = torch.randn(B, T, D)
mask = torch.tensor([
    [1, 1, 1, 0, 0],   # first sequence length 3
    [0, 0, 0, 0, 0],   # second is fully padded
])

m = batch_safe_mean(x, mask, default=0.0)
print(m.shape)   # (2, 3)
# row 0: mean over first 3 time steps
# row 1: [0., 0., 0.] from default

import numpy as np

def np_safe_mean(x: np.ndarray, axis=None, keepdims=False):
  if np.size(x) == 0:
      # Empty → return 0 with requested shape
      if axis is None:
          return 0.0
      # build shape as if mean had been taken, but filled with 0
      return np.zeros(np.mean(x, axis=axis, keepdims=keepdims).shape, dtype=float)

  # normal mean is fine when non-empty
  return np.mean(x, axis=axis, keepdims=keepdims)



def np_safe_masked_mean(x: np.ndarray,
                        mask: np.ndarray,
                        axis=None,
                        keepdims=False):
    m = mask.astype(float)
    masked = x * m

    num = masked.sum(axis=axis, keepdims=keepdims)
    count = m.sum(axis=axis, keepdims=keepdims)
    safe_count = np.clip(count, 1.0, None)
    return num / safe_count



def np_safe_segment_mean(values: np.ndarray,
                         segment_ids: np.ndarray,
                         num_segments: int | None = None):
    if num_segments is None:
        num_segments = int(segment_ids.max()) + 1

    rest_shape = values.shape[1:]
    sums = np.zeros((num_segments, *rest_shape), dtype=values.dtype)
    counts = np.zeros(num_segments, dtype=float)

    np.add.at(sums, segment_ids, values)
    np.add.at(counts, segment_ids, 1.0)

    counts = np.clip(counts, 1.0, None)
    # reshape for broadcast
    while counts.ndim < sums.ndim:
        counts = counts[..., None]

    return sums / counts




In [ ]:
# triton_safe_masked_mean.py
import torch
import triton
import triton.language as tl

@triton.jit
def safe_masked_mean_rowwise_kernel(
    x_ptr,         # *f32
    mask_ptr,      # *i32 or *f32 (0/1); pass nullptr for unmasked
    out_ptr,       # *f32
    B, N,
    stride_xb, stride_xn,
    stride_mb, stride_mn,
    BLOCK_N: tl.constexpr,
):
    b = tl.program_id(0)  # batch index

    offs_n = tl.arange(0, BLOCK_N)
    row_x_ptr = x_ptr + b * stride_xb + offs_n * stride_xn

    has_mask = mask_ptr != 0
    if has_mask:
        row_m_ptr = mask_ptr + b * stride_mb + offs_n * stride_mn

    acc_sum = tl.zeros((), dtype=tl.float32)
    acc_count = tl.zeros((), dtype=tl.float32)

    for start_n in range(0, N, BLOCK_N):
        cur_mask = start_n + offs_n < N

        x_vals = tl.load(row_x_ptr + start_n * stride_xn,
                         mask=cur_mask,
                         other=0.0)

        if has_mask:
            m_vals = tl.load(row_m_ptr + start_n * stride_mn,
                             mask=cur_mask,
                             other=0)
            # assume mask is 0/1 or bool, cast to float
            m_vals_f = m_vals.to(tl.float32)
        else:
            m_vals_f = tl.where(cur_mask, 1.0, 0.0)

        acc_sum += tl.sum(x_vals * m_vals_f, axis=0)
        acc_count += tl.sum(m_vals_f, axis=0)

    # safe mean: if acc_count == 0, define mean = 0
    safe_count = tl.maximum(acc_count, 1.0)
    mean = acc_sum / safe_count
    mean = tl.where(acc_count > 0, mean, 0.0)

    tl.store(out_ptr + b, mean)

def safe_masked_mean_rowwise(x: torch.Tensor, mask: torch.Tensor | None = None):
    """
    x: (B, N), float32
    mask: (B, N) or None; 0/1 or bool
    returns: (B,)
    """
    assert x.dim() == 2
    B, N = x.shape
    x = x.contiguous()

    if mask is not None:
        mask = mask.to(torch.int32).contiguous()
        mask_ptr = mask
    else:
        mask_ptr = torch.tensor([], device=x.device, dtype=torch.int32)  # dummy
        # We'll treat 'mask_ptr != 0' as 'has_mask', so ensure it's not literally None.
        # Instead: we pass mask_ptr.data_ptr()==0? Triton can't; hack: pass 0 below.

    out = torch.empty(B, device=x.device, dtype=torch.float32)

    BLOCK_N = 128
    grid = (B,)

    # carefully pass 0 for mask_ptr when mask is None
    mask_arg = mask_ptr if mask is not None else 0

    safe_masked_mean_rowwise_kernel[grid](
        x,
        mask_arg,
        out,
        B, N,
        x.stride(0), x.stride(1),
        mask.stride(0) if mask is not None else 0,
        mask.stride(1) if mask is not None else 0,
        BLOCK_N=BLOCK_N,
    )
    return out

// safe_masked_mean.cu
#include <cuda_runtime.h>
#include <stdint.h>

__global__ void safe_masked_mean_kernel(
    const float* __restrict__ x,
    const uint8_t* __restrict__ mask,  // 0 or 1; nullptr for unmasked
    int64_t N,
    float* __restrict__ out_sum,
    float* __restrict__ out_count
) {
    extern __shared__ float shmem[];
    float* sh_sum   = shmem;
    float* sh_count = shmem + blockDim.x;

    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    float local_sum = 0.0f;
    float local_count = 0.0f;

    for (int64_t i = idx; i < N; i += blockDim.x * gridDim.x) {
        uint8_t m = mask ? mask[i] : 1;
        if (m) {
            local_sum += x[i];
            local_count += 1.0f;
        }
    }

    sh_sum[threadIdx.x] = local_sum;
    sh_count[threadIdx.x] = local_count;
    __syncthreads();

    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (threadIdx.x < stride) {
            sh_sum[threadIdx.x] += sh_sum[threadIdx.x + stride];
            sh_count[threadIdx.x] += sh_count[threadIdx.x + stride];
        }
        __syncthreads();
    }

    if (threadIdx.x == 0) {
        atomicAdd(out_sum, sh_sum[0]);
        atomicAdd(out_count, sh_count[0]);
    }
}


#include <torch/extension.h>  // or your own wrapper
#include <cuda_runtime.h>

std::pair<float, float> safe_masked_mean_cuda(
    const float* d_x,
    const uint8_t* d_mask,
    int64_t N
) {
    float h_sum = 0.0f;
    float h_count = 0.0f;

    float* d_sum;
    float* d_count;
    cudaMalloc(&d_sum, sizeof(float));
    cudaMalloc(&d_count, sizeof(float));
    cudaMemcpy(d_sum, &h_sum, sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_count, &h_count, sizeof(float), cudaMemcpyHostToDevice);

    int threads = 256;
    int blocks = (N + threads - 1) / threads;
    size_t shmem = 2 * threads * sizeof(float);

    safe_masked_mean_kernel<<<blocks, threads, shmem>>>(d_x, d_mask, N, d_sum, d_count);

    cudaMemcpy(&h_sum, d_sum, sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(&h_count, d_count, sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(d_sum);
    cudaFree(d_count);

    // safe mean
    float mean = (h_count > 0.0f) ? (h_sum / h_count) : 0.0f;
    return {mean, h_count};
}

sum[seg] = Σ x[i] for i with segment_ids[i] == seg and mask[i]==1
count[seg] = Σ 1    for same
mean[seg] = sum[seg] / max(count[seg], 1)

// segment_mean.cu
#include <cuda_runtime.h>
#include <stdint.h>

__global__ void segment_sum_count_kernel(
    const float* __restrict__ x,
    const int32_t* __restrict__ segment_ids,
    const uint8_t* __restrict__ mask,    // 0/1 or nullptr
    int64_t N,
    float* __restrict__ seg_sums,
    float* __restrict__ seg_counts,
    int32_t num_segments
) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    for (int64_t i = idx; i < N; i += blockDim.x * gridDim.x) {
        uint8_t m = mask ? mask[i] : 1;
        if (!m) {
            continue;
        }

        int32_t seg = segment_ids[i];
        if (seg < 0 || seg >= num_segments) {
            continue;  // or assert
        }

        float val = x[i];
        atomicAdd(&seg_sums[seg], val);
        atomicAdd(&seg_counts[seg], 1.0f);
    }
}

__global__ void segment_safe_mean_kernel(
    const float* __restrict__ seg_sums,
    const float* __restrict__ seg_counts,
    float* __restrict__ seg_means,
    int32_t num_segments
) {
    int seg = blockIdx.x * blockDim.x + threadIdx.x;
    if (seg >= num_segments) return;

    float s = seg_sums[seg];
    float c = seg_counts[seg];

    if (c > 0.0f) {
        seg_means[seg] = s / c;
    } else {
        seg_means[seg] = 0.0f;  // safe mean for empty segment
    }
}

#include <torch/extension.h>  // or your own wrapper
#include <cuda_runtime.h>

std::pair<float, float> safe_masked_mean_cuda(
    const float* d_x,
    const uint8_t* d_mask,
    int64_t N
) {
    float h_sum = 0.0f;
    float h_count = 0.0f;

    float* d_sum;
    float* d_count;
    cudaMalloc(&d_sum, sizeof(float));
    cudaMalloc(&d_count, sizeof(float));
    cudaMemcpy(d_sum, &h_sum, sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_count, &h_count, sizeof(float), cudaMemcpyHostToDevice);

    int threads = 256;
    int blocks = (N + threads - 1) / threads;
    size_t shmem = 2 * threads * sizeof(float);

    safe_masked_mean_kernel<<<blocks, threads, shmem>>>(d_x, d_mask, N, d_sum, d_count);

    cudaMemcpy(&h_sum, d_sum, sizeof(float), cudaMemcpyDeviceToHost);
    cudaMemcpy(&h_count, d_count, sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(d_sum);
    cudaFree(d_count);

    // safe mean
    float mean = (h_count > 0.0f) ? (h_sum / h_count) : 0.0f;
    return {mean, h_count};
}

#sum[seg] = Σ x[i] for i with segment_ids[i] == seg and mask[i]==1
#count[seg] = Σ 1    for same
#mean[seg] = sum[seg] / max(count[seg], 1)

// segment_mean.cu
#include <cuda_runtime.h>
#include <stdint.h>

__global__ void segment_sum_count_kernel(
    const float* __restrict__ x,
    const int32_t* __restrict__ segment_ids,
    const uint8_t* __restrict__ mask,    // 0/1 or nullptr
    int64_t N,
    float* __restrict__ seg_sums,
    float* __restrict__ seg_counts,
    int32_t num_segments
) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    for (int64_t i = idx; i < N; i += blockDim.x * gridDim.x) {
        uint8_t m = mask ? mask[i] : 1;
        if (!m) {
            continue;
        }

        int32_t seg = segment_ids[i];
        if (seg < 0 || seg >= num_segments) {
            continue;  // or assert
        }

        float val = x[i];
        atomicAdd(&seg_sums[seg], val);
        atomicAdd(&seg_counts[seg], 1.0f);
    }
}

__global__ void segment_safe_mean_kernel(
    const float* __restrict__ seg_sums,
    const float* __restrict__ seg_counts,
    float* __restrict__ seg_means,
    int32_t num_segments
) {
    int seg = blockIdx.x * blockDim.x + threadIdx.x;
    if (seg >= num_segments) return;

    float s = seg_sums[seg];
    float c = seg_counts[seg];

    if (c > 0.0f) {
        seg_means[seg] = s / c;
    } else {
        seg_means[seg] = 0.0f;  // safe mean for empty segment
    }
}

void segment_mean_cuda(
    const float* d_x,
    const int32_t* d_segment_ids,
    const uint8_t* d_mask,      // may be nullptr
    int64_t N,
    int32_t num_segments,
    float* d_out_means
) {
    float* d_sums;
    float* d_counts;
    cudaMalloc(&d_sums,   num_segments * sizeof(float));
    cudaMalloc(&d_counts, num_segments * sizeof(float));
    cudaMemset(d_sums,   0, num_segments * sizeof(float));
    cudaMemset(d_counts, 0, num_segments * sizeof(float));

    int threads = 256;
    int blocks = (N + threads - 1) / threads;

    segment_sum_count_kernel<<<blocks, threads>>>(
        d_x, d_segment_ids, d_mask, N, d_sums, d_counts, num_segments
    );

    int blocks_seg = (num_segments + threads - 1) / threads;
    segment_safe_mean_kernel<<<blocks_seg, threads>>>(
        d_sums, d_counts, d_out_means, num_segments
    );

    cudaFree(d_sums);
    cudaFree(d_counts);
}


#triton segment mean sketch
@triton.jit
def segment_sum_count_kernel_triton(
    x_ptr, seg_id_ptr, mask_ptr,
    seg_sums_ptr, seg_counts_ptr,
    N, NUM_SEGMENTS: tl.constexpr,
    BLOCK_SIZE: tl.constexpr,
):
    pid = tl.program_id(0)
    offs = pid * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)
    mask = offs < N

    x = tl.load(x_ptr + offs, mask=mask, other=0.0)
    seg = tl.load(seg_id_ptr + offs, mask=mask, other=0)
    has_mask = mask_ptr != 0
    if has_mask:
        m = tl.load(mask_ptr + offs, mask=mask, other=0)
        valid = mask & (m != 0)
    else:
        valid = mask

    x = tl.where(valid, x, 0.0)
    seg = tl.where(valid, seg, 0)

    # atomic adds
    tl.atomic_add(seg_sums_ptr + seg, x, mask=valid)
    tl.atomic_add(seg_counts_ptr + seg,
                  tl.where(valid, 1.0, 0.0),
                  mask=valid)

In [ ]:
# Colab-ready benchmark: safe mean, masked mean, segment mean, batch mean
# Across: PyTorch CPU, PyTorch GPU, NumPy CPU, Triton kernel, CUDA (CuPy) kernel
import torch
!pip install -q triton==3.0.0 cupy-cuda12x

import time
import math
from dataclasses import dataclass

import numpy as np
import torch
import torch.nn.functional as F

import triton
import triton.language as tl

import cupy as cp

# ------------------------------------------------------------
# 0. Config
# ------------------------------------------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

# Problem sizes
BATCH = 4096     # number of rows
N      = 1024    # row length
SEG_K  = 128     # number of segments for segment mean

WARMUP_ITERS = 5
BENCH_ITERS  = 20

# ------------------------------------------------------------
# 1. Safe mean helpers (PyTorch & NumPy)
# ------------------------------------------------------------

def torch_safe_mean(x: torch.Tensor, dim=None, keepdim=False) -> torch.Tensor:
    """
    Safe mean: if count == 0, returns 0 (no NaN).
    Works with arbitrary dimension, keeps gradients.
    """
    if dim is None:
        # flatten
        x_flat = x.view(-1)
        count = x_flat.numel()
        if count == 0:
            return x_flat.new_zeros(())
        return x_flat.sum() / max(count, 1)
    else:
        # general dim
        x = x.float()
        ones = torch.ones_like(x, dtype=x.dtype)
        count = ones.sum(dim=dim, keepdim=keepdim)
        s = x.sum(dim=dim, keepdim=keepdim)
        # clamp denominator to at least 1 to avoid NaNs; where count==0, we force 0
        denom = count.clamp_min(1.0)
        mean = s / denom
        mean = torch.where(count > 0, mean, torch.zeros_like(mean))
        return mean

def torch_masked_mean(x: torch.Tensor, mask: torch.Tensor, dim=-1, keepdim=False):
    """
    mask: bool or 0/1, same shape as x.
    """
    x = x.float()
    mask = mask.to(dtype=x.dtype)
    s = (x * mask).sum(dim=dim, keepdim=keepdim)
    count = mask.sum(dim=dim, keepdim=keepdim)
    denom = count.clamp_min(1.0)
    mean = s / denom
    mean = torch.where(count > 0, mean, torch.zeros_like(mean))
    return mean

def torch_segment_mean(x: torch.Tensor, segment_ids: torch.Tensor, num_segments: int):
    """
    x: [N, D] or [N]; segment_ids: [N] in [0, num_segments-1]
    Returns [num_segments, D] or [num_segments]
    """
    if x.dim() == 1:
        x = x[:, None]
        squeeze = True
    else:
        squeeze = False

    N, D = x.shape
    device = x.device

    segment_ids = segment_ids.to(device=device, dtype=torch.long)
    out = torch.zeros(num_segments, D, device=device, dtype=x.dtype)
    count = torch.zeros(num_segments, 1, device=device, dtype=x.dtype)

    out.index_add_(0, segment_ids, x)
    ones = torch.ones(N, 1, device=device, dtype=x.dtype)
    count.index_add_(0, segment_ids, ones)

    denom = count.clamp_min(1.0)
    mean = out / denom
    mean = torch.where(count > 0, mean, torch.zeros_like(mean))

    if squeeze:
        mean = mean[:, 0]
    return mean

# NumPy equivalents (no gradient)
def np_safe_mean(x: np.ndarray, axis=None, keepdims=False):
    if x.size == 0:
        return np.zeros((), dtype=x.dtype)
    count = x.shape[axis] if axis is not None else x.size
    s = x.sum(axis=axis, keepdims=keepdims)
    return s / max(count, 1)

def np_masked_mean(x: np.ndarray, mask: np.ndarray, axis=-1, keepdims=False):
    x = x.astype(np.float32)
    mask = mask.astype(np.float32)
    s = (x * mask).sum(axis=axis, keepdims=keepdims)
    count = mask.sum(axis=axis, keepdims=keepdims)
    denom = np.clip(count, 1.0, None)
    out = s / denom
    out = np.where(count > 0, out, np.zeros_like(out))
    return out

import numpy as np

def np_segment_mean(x: np.ndarray,
                    segment_ids: np.ndarray,
                    num_segments: int):
    # Handle 1D x by temporarily promoting to 2D
    if x.ndim == 1:
        x = x[:, None]   # (N,) -> (N, 1)
        squeeze = True
    else:
        squeeze = False

    N, D = x.shape
    out = np.zeros((num_segments, D), dtype=x.dtype)      # sum per segment
    count = np.zeros((num_segments,), dtype=np.int64)     # count per segment

    # Accumulate per segment
    for i in range(N):
        seg = int(segment_ids[i])
        if 0 <= seg < num_segments:
            out[seg] += x[i]
            count[seg] += 1

    # Avoid div-by-zero by clamping denominator to at least 1
    denom = np.maximum(count, 1).reshape(num_segments, 1)  # (K, 1)
    mean = out / denom                                    # (K, D)

    # For segments where count == 0, explicitly set mean to 0
    zero_mask = (count == 0)  # (K,)
    mean[zero_mask] = 0.0

    if squeeze:
        mean = mean[:, 0]  # (K,)
    return mean

# ------------------------------------------------------------
# 2. Data setup
# ------------------------------------------------------------
torch.manual_seed(0)
np.random.seed(0)

x_torch_cpu = torch.randn(BATCH, N, dtype=torch.float32)
mask_torch_cpu = (torch.rand(BATCH, N) > 0.3).to(torch.bool)

seg_ids_cpu = torch.randint(0, SEG_K, (BATCH,), dtype=torch.long)

x_np = x_torch_cpu.numpy()
mask_np = mask_torch_cpu.numpy().astype(np.bool_)
seg_ids_np = seg_ids_cpu.numpy()

if DEVICE == "cuda":
    x_torch_gpu = x_torch_cpu.to("cuda")
    mask_torch_gpu = mask_torch_cpu.to("cuda")
    seg_ids_gpu = seg_ids_cpu.to("cuda")
else:
    x_torch_gpu = None
    mask_torch_gpu = None
    seg_ids_gpu = None

# ------------------------------------------------------------
# 3. Triton kernel: row-wise mean (batch mean) for 2D tensor
# ------------------------------------------------------------

@triton.jit
def row_mean_kernel(X_ptr, Y_ptr, BATCH, N, BLOCK_SIZE: tl.constexpr):
    row_id = tl.program_id(0)
    offs = row_id * N + tl.arange(0, BLOCK_SIZE)
    mask = offs < (row_id * N + N)

    x = tl.load(X_ptr + offs, mask=mask, other=0.0)
    # parallel reduction in block
    # here we just sum and rely on BLOCK_SIZE == N for simplicity
    # (you can generalize to partial tiles if needed)
    s = tl.sum(x, axis=0)
    # each program handles a whole row
    denom = N
    mean = s / denom
    tl.store(Y_ptr + row_id, mean)

def triton_row_mean(x: torch.Tensor) -> torch.Tensor:
    """
    x: [B, N] on CUDA
    returns: [B]
    """
    assert x.is_cuda
    B, N = x.shape
    y = torch.empty(B, device=x.device, dtype=x.dtype)

    BLOCK_SIZE = N  # simple case: one block per row
    grid = (B,)

    row_mean_kernel[grid](
        x, y,
        BATCH=B,
        N=N,
        BLOCK_SIZE=BLOCK_SIZE,
        num_warps=4,
    )
    return y

# ------------------------------------------------------------
# 4. CUDA kernel with CuPy: row-wise mean
# ------------------------------------------------------------

cuda_row_mean_src = r"""
extern "C" __global__
void row_mean(const float* __restrict__ x,
              float* __restrict__ y,
              int B, int N) {
    int row = blockIdx.x;
    if (row >= B) return;

    float sum = 0.0f;
    for (int i = threadIdx.x; i < N; i += blockDim.x) {
        sum += x[row * N + i];
    }

    __shared__ float smem[256]; // up to 256 threads
    int tid = threadIdx.x;
    smem[tid] = sum;
    __syncthreads();

    // simple reduction in shared memory
    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (tid < stride) {
            smem[tid] += smem[tid + stride];
        }
        __syncthreads();
    }

    if (tid == 0) {
        y[row] = smem[0] / (float)N;
    }
}
"""

row_mean_kernel_cuda = cp.RawKernel(cuda_row_mean_src, "row_mean")

def cuda_row_mean(x_torch: torch.Tensor) -> torch.Tensor:
    """
    Takes a CUDA torch tensor [B, N], uses CuPy to run custom kernel,
    returns torch tensor [B].
    """
    assert x_torch.is_cuda
    B, N = x_torch.shape
    # zero-copy via DLPack
    x_cu = cp.fromDlpack(torch.utils.dlpack.to_dlpack(x_torch))
    y_cu = cp.empty((B,), dtype=cp.float32)

    threads_per_block = 256
    blocks = (B,)

    row_mean_kernel_cuda(blocks, (threads_per_block,),
                         (x_cu, y_cu, B, N))

    # back to torch
    y_torch = torch.utils.dlpack.from_dlpack(y_cu.toDlpack())
    return y_torch

# ------------------------------------------------------------
# 5. Benchmark helpers
# ------------------------------------------------------------

@dataclass
class BenchResult:
    name: str
    time_ms: float

def bench(fn, iters=BENCH_ITERS, warmup=WARMUP_ITERS):
    # Warmup
    for _ in range(warmup):
        out = fn()
        if isinstance(out, torch.Tensor) and out.is_cuda:
            torch.cuda.synchronize()

    t0 = time.perf_counter()
    for _ in range(iters):
        out = fn()
        if isinstance(out, torch.Tensor) and out.is_cuda:
            torch.cuda.synchronize()
    t1 = time.perf_counter()
    return (t1 - t0) * 1000.0 / iters

results = []

# ------------------------------------------------------------
# 6. PyTorch CPU benchmarks
# ------------------------------------------------------------
print("\n=== PyTorch CPU ===")

# mean (batch-level: row-wise)
def fn_torch_cpu_batch_mean():
    return torch_safe_mean(x_torch_cpu, dim=1)  # [B]

t = bench(fn_torch_cpu_batch_mean)
results.append(BenchResult("torch_cpu_batch_mean", t))
print("torch_cpu_batch_mean: %.3f ms" % t)

# masked mean (row-wise)
def fn_torch_cpu_masked_mean():
    return torch_masked_mean(x_torch_cpu, mask_torch_cpu, dim=1)

t = bench(fn_torch_cpu_masked_mean)
results.append(BenchResult("torch_cpu_masked_mean", t))
print("torch_cpu_masked_mean: %.3f ms" % t)

# segment mean (over batch dimension)
def fn_torch_cpu_segment_mean():
    return torch_segment_mean(x_torch_cpu, seg_ids_cpu, num_segments=SEG_K)

t = bench(fn_torch_cpu_segment_mean)
results.append(BenchResult("torch_cpu_segment_mean", t))
print("torch_cpu_segment_mean: %.3f ms" % t)

# ------------------------------------------------------------
# 7. PyTorch GPU benchmarks (if available)
# ------------------------------------------------------------
if DEVICE == "cuda":
    print("\n=== PyTorch GPU ===")

    def fn_torch_gpu_batch_mean():
        return torch_safe_mean(x_torch_gpu, dim=1)

    t = bench(fn_torch_gpu_batch_mean)
    results.append(BenchResult("torch_gpu_batch_mean", t))
    print("torch_gpu_batch_mean: %.3f ms" % t)

    def fn_torch_gpu_masked_mean():
        return torch_masked_mean(x_torch_gpu, mask_torch_gpu, dim=1)

    t = bench(fn_torch_gpu_masked_mean)
    results.append(BenchResult("torch_gpu_masked_mean", t))
    print("torch_gpu_masked_mean: %.3f ms" % t)

    def fn_torch_gpu_segment_mean():
        return torch_segment_mean(x_torch_gpu, seg_ids_gpu, num_segments=SEG_K)

    t = bench(fn_torch_gpu_segment_mean)
    results.append(BenchResult("torch_gpu_segment_mean", t))
    print("torch_gpu_segment_mean: %.3f ms" % t)

# ------------------------------------------------------------
# 8. NumPy CPU benchmarks
# ------------------------------------------------------------
print("\n=== NumPy CPU ===")

def fn_numpy_batch_mean():
    return np_safe_mean(x_np, axis=1)

t = bench(fn_numpy_batch_mean)
results.append(BenchResult("numpy_batch_mean", t))
print("numpy_batch_mean: %.3f ms" % t)

def fn_numpy_masked_mean():
    return np_masked_mean(x_np, mask_np, axis=1)

t = bench(fn_numpy_masked_mean)
results.append(BenchResult("numpy_masked_mean", t))
print("numpy_masked_mean: %.3f ms" % t)

def fn_numpy_segment_mean():
    return np_segment_mean(x_np, seg_ids_np, num_segments=SEG_K)

t = bench(fn_numpy_segment_mean)
results.append(BenchResult("numpy_segment_mean", t))
print("numpy_segment_mean: %.3f ms" % t)

# ------------------------------------------------------------
# 9. Triton benchmark (row-wise mean only)
# ------------------------------------------------------------
if DEVICE == "cuda":
    print("\n=== Triton row-wise mean (CUDA) ===")

    def fn_triton_row_mean():
        return triton_row_mean(x_torch_gpu)

    t = bench(fn_triton_row_mean)
    results.append(BenchResult("triton_row_mean", t))
    print("triton_row_mean: %.3f ms" % t)

# ------------------------------------------------------------
# 10. CUDA (CuPy) kernel benchmark (row-wise mean only)
# ------------------------------------------------------------
if DEVICE == "cuda":
    print("\n=== CUDA (CuPy) row-wise mean ===")

    def fn_cuda_row_mean():
        return cuda_row_mean(x_torch_gpu)

    t = bench(fn_cuda_row_mean)
    results.append(BenchResult("cuda_row_mean", t))
    print("cuda_row_mean: %.3f ms" % t)

# ------------------------------------------------------------
# 11. Summary
# ------------------------------------------------------------
print("\n=== Summary (ms per call) ===")
for r in results:
    print(f"{r.name:30s}: {r.time_ms:8.3f} ms")

Math trick in self attention
Karpathy YT https://www.youtube.com/watch?v=kCc8FmEb1nY 42:27


In [ ]:

torch.manual_seed(1337)
B, T, C = 4,8,2
x = torch.randn(B,T,C)
x.shape
#we want to look at past tokens from currnt positio
# Batch, Time, Channels
#simplest way to communicate with past tokens is to take average of tokens before
# current token. This vector of 5 past tokens with an average becomes
#
xbow = torch.zeros(B,T,C)
for batch in range(B):
  for time in range(T):
    xprev = x[batch, time+1, ] #t,C
    xbow[b,t] = torch.mean()
